<a href="https://colab.research.google.com/github/DmitryAlexandrovv/infopoisk/blob/master/task3/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 41.6 MB/s eta 0:00:00


In [30]:
!python -m spacy download ru_core_news_sm

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-04-05 15:29:54.973618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 15:29:58.256639: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-05 15:30:01.930868: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 3.4 MB/

In [29]:
!pip install spacy_langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.1/998.1 KB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=2ef9784ffab81c9d9077febb2c7d00f36f6ee30afc022c2dc3ca659417f46854
  Stored in directory: /root/.cache/pip/wheels/4a/03/92/e3a289314c0c6a8e41c67b88a95798b1cc238fba986d2cfe51
Successfully built langdetect


In [37]:
from bs4 import BeautifulSoup
import pymorphy2
from spacy_langdetect import LanguageDetector
import spacy
import string
from google.colab import drive
drive.mount('/content/drive')

def get_text_from_html(html):
    soup = BeautifulSoup(html, features="html.parser")

    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text(' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

def pos(word, morth=pymorphy2.MorphAnalyzer()):
    return morth.parse(word)[0].tag.POS

def get_lang_detector(nlp, name):
    return LanguageDetector()

load_model = spacy.load('ru_core_news_sm')
posled = []

for i in range(100):
    with open('drive/MyDrive/dz/pages/page' + str(i) + '.html', 'r') as file:
        text = get_text_from_html(file.read())

        # Убираем предлоги и союзы, знаки препинания
        words = text.split()
        functors_pos = {'CONJ', 'PREP'}  # function words
        tt = str.maketrans(dict.fromkeys(string.punctuation))
        prepared_text = ' '.join([word for word in words if pos(word) not in functors_pos]).translate(tt)

        doc = load_model(prepared_text.lower())

        if len(str(token)) > 0:
            for token in doc:
                posled.append({ str(token): i })

res = {}

for item in posled:
    for value in item:
        if res.get(value):
            res.update({ value: res.get(value) + ' ' + str(item.get(value)) })
        else:
            res.update({ value: str(item.get(value)) })

with open('drive/MyDrive/dz/invert_index.txt', 'w') as writefile:
    for value in res:
        writefile.write(str(value) + ': ' + res.get(value) + '\n')

def search(first_word, second_word, operator):
    first_word_insertions = res.get(first_word)
    second_word_insertions = res.get(second_word)

    search_res = []

    if (first_word_insertions and second_word_insertions):
        first_word_docs = first_word_insertions.split(' ')
        second_word_docs = second_word_insertions.split(' ')

        if operator == 'or':
            for doc in first_word_docs:
                if doc not in search_res:
                    search_res.append(doc)
            for doc in second_word_docs:
                if doc not in search_res:
                    search_res.append(doc)
        if operator == 'and':
            for doc in first_word_docs:
                if doc not in search_res and doc in second_word_docs:
                    search_res.append(doc)
            for doc in second_word_docs:
                if doc not in search_res and doc in first_word_docs:
                    search_res.append(doc)
        if operator == 'not':
            for doc in range(100):
                if str(doc) not in first_word_docs and str(doc) not in second_word_docs:
                     search_res.append(doc)

    return search_res

print(search('советы', 'лайфхаки', 'and'))
print(search('советы', 'лайфхаки', 'or'))
print(search('советы', 'лайфхаки', 'not'))
        


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['0', '64']
['0', '1', '2', '6', '23', '29', '30', '37', '38', '47', '48', '51', '53', '64']
[3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
